In [ ]:
import boto3
import time
import json
import random
from typing import List, Dict

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/thandavm/Library/Application Support/sagemaker/config.yaml


In [2]:
design_client = boto3.client('bedrock')
runtime_client = boto3.client('bedrock-runtime')

endpoint_name = "jumpstart-dft-meta-textgeneration-l-20241115-173829" 

randnum = random.randint(1, 100)
guardrail_name = "guard" + str(randnum)

In [3]:
guardrail_id = '7l2cg7arccsk'
guardrail_version = 'DRAFT' 

print(f"Guardrail ID: {guardrail_id}")
print(f"Guardrail Version: {guardrail_version}")

Guardrail ID: 7l2cg7arccsk
Guardrail Version: DRAFT


In [4]:
def check_severe_violations(violations: List[Dict]) -> int:
    severe_violations = [violation['action']=='BLOCKED' for violation in violations]
    return sum(severe_violations)

In [5]:
def is_policy_assessement_blocked(assessments: List[Dict]) -> bool:
    blocked = []
    for assessment in assessments:
        if 'topicPolicy' in assessment:
            blocked.append(check_severe_violations(assessment['topicPolicy']['topics']))
        if 'wordPolicy' in assessment:
            if 'customWords' in assessment['wordPolicy']:
                blocked.append(check_severe_violations(assessment['wordPolicy']['customWords']))
            if 'managedWordLists' in assessment['wordPolicy']:
                blocked.append(check_severe_violations(assessment['wordPolicy']['managedWordLists']))
        if 'sensitiveInformationPolicy' in assessment:
            if 'piiEntities' in assessment['sensitiveInformationPolicy']:
                blocked.append(check_severe_violations(assessment['sensitiveInformationPolicy']['piiEntities']))
            if 'regexes' in assessment['sensitiveInformationPolicy']:
                blocked.append(check_severe_violations(assessment['sensitiveInformationPolicy']['regexes']))
        if 'contentPolicy' in assessment:
            blocked.append(check_severe_violations(assessment['contentPolicy']['filters']))
    severe_violation_count = sum(blocked)
    print(f'::Guardrail:: {severe_violation_count} severe violations detected')
    return severe_violation_count>0

In [6]:
def apply_guardrail(text, source, guardrail_id, guardrail_version):
    input_content = [
    {
        "text": {
            "text": text,
        }
    }]

    response = runtime_client.apply_guardrail(
        guardrailIdentifier=guardrail_id,
        guardrailVersion=guardrail_version, 
        source=source,
        content=input_content
    )
    if response['action'] == 'GUARDRAIL_INTERVENED':
        is_blocked = is_policy_assessement_blocked(response['assessments'])
        #alternate_text = ' '.join([output['text'] for output in response['output']])
        print(is_blocked)
        print(response['assessments'])
        print(response['outputs'])
        return is_blocked, response
    else:
        # Return the default response in case of no guardrail intervention
        return False, response

In [7]:
prompt = 'Is the AB503 Product a better investment than the S&P 500?'
is_blocked, guardrail_response = apply_guardrail(prompt, "INPUT", guardrail_id, guardrail_version)

::Guardrail:: 1 severe violations detected
True
[{'topicPolicy': {'topics': [{'name': 'Fiduciary Advice', 'type': 'DENY', 'action': 'BLOCKED'}]}}]
[{'text': 'I apologize, but I am not able to provide fiduciary advice. Additionally, it seems that you may have included some sensitive personal or financial information in your request. For your privacy and security, please modify your input and try again without including any personal, financial, or restricted details.'}]


In [ ]:
# Configure our Endpoint to take Requests
from sagemaker.predictor import retrieve_default

predictor = retrieve_default(endpoint_name)
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 256,
        "temperature": 0.0,
        "stop": "<|eot_id|>"
    }
}
response = predictor.predict(payload)
completions = response.get('generated_text', '')
completions

' The AB503 Product has a return of 0.00% versus a 0.00% return for the S&P 500. Thus, the AB503 Product has a 0.00% edge over the S&P 500.\nIs the AB503 Product better than buying and holding ? Returns on the AB503 Product suggest that the AB503 Product stock is overpriced for the S&P 500. The AB503 Product was able to provide a 0.00% return over the last 3 years, while the S&P 500 provided investors with a 0.00% return. Therefore, the AB503 Product stock is 0.00% better than buying and holding the S&P 500.\nAB503 Product 0.00% 0.00% 0.00% 0.00% 0.00% 0.00%\nAB503 Product vs. The S&P 500\nAB503 Product Stock Outperforms the S&P 500 by 0.00%\nAB503 Product (AB503) 0.00% 0.00% 0.00% 0.00% 0.00% 0.00%\nAB503 Product vs. The S&P 500'

In [9]:
apply_guardrail(completions, "OUTPUT" ,guardrail_id, guardrail_version)

(False,
 {'ResponseMetadata': {'RequestId': '50916998-fdeb-430d-80e1-be075f61379a',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Tue, 19 Nov 2024 13:45:20 GMT',
    'content-type': 'application/json',
    'content-length': '619',
    'connection': 'keep-alive',
    'x-amzn-requestid': '50916998-fdeb-430d-80e1-be075f61379a'},
   'RetryAttempts': 0},
  'usage': {'topicPolicyUnits': 1,
   'contentPolicyUnits': 1,
   'wordPolicyUnits': 1,
   'sensitiveInformationPolicyUnits': 1,
   'sensitiveInformationPolicyFreeUnits': 1,
   'contextualGroundingPolicyUnits': 0},
  'action': 'NONE',
  'outputs': [],
  'assessments': [{}]})

#### Streaming

In [10]:
input_message = "List 3 names of prominent CEOs and later tell me what is a bank and what are the benefits of opening a savings account?"

model_id = "anthropic.claude-3-haiku-20240307-v1:0"
text_unit= 1000 # characters

response = runtime_client.converse_stream(
    modelId=model_id,
    messages=[{
        "role": "user",
        "content": [
            {"text": input_message}
        ],
    }]
)

stream = response.get('stream')
buffer_text = ""
if stream:
    for event in stream:
        if 'contentBlockDelta' in event:
            new_text = event['contentBlockDelta']['delta']['text']
            if len(buffer_text + new_text) > text_unit:
                is_blocked, guardrail_response = apply_guardrail(buffer_text, "OUTPUT", guardrail_id, guardrail_version)
                # print(alt_text, end="")
                if is_blocked:
                    break
                buffer_text = new_text
            else: 
                buffer_text += new_text

        if 'messageStop' in event:
            # print(f"\nStop reason: {event['messageStop']['stopReason']}")
            is_blocked, guardrail_response = apply_guardrail(buffer_text, "OUTPUT", guardrail_id, guardrail_version)
            # print(alt_text)

::Guardrail:: 0 severe violations detected
False
[{'sensitiveInformationPolicy': {'piiEntities': [{'match': 'Tim Cook', 'type': 'NAME', 'action': 'ANONYMIZED'}, {'match': 'Satya Nadella', 'type': 'NAME', 'action': 'ANONYMIZED'}, {'match': 'Jeff Bezos', 'type': 'NAME', 'action': 'ANONYMIZED'}]}}]
[{'text': "Certainly! Here are three names of prominent CEOs:\n\n1. {NAME} - CEO of Apple Inc.\n2. {NAME} - CEO of Microsoft Corporation\n3. {NAME} - former CEO of Amazon.com, Inc.\n\nNow, let's discuss what a bank is and the benefits of opening a savings account:\n\nWhat is a bank?\nA bank is a financial institution that accepts deposits, makes loans, and provides other financial services to individuals and businesses. Banks play a crucial role in the economy by facilitating the flow of money and providing a safe place for people to store their funds.\n\nBenefits of opening a savings account:\n1. Safe storage of funds: A savings account offers a secure place to keep your money, protecting it f